In [1]:
import son_main_script as soon
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from pymoo.indicators.hv import HV
import json
import os
import numpy as np

In [6]:
# load dataset

network_name = "hetNet"
configuration_name_list =[ "algorithm_config_1"]

for configuration_name in configuration_name_list:
    results_directory = "./datastore/{network_name}/{configuration_name}/".format(network_name=network_name, configuration_name= configuration_name)

    with open(results_directory + "/objectives_result.json", mode="r",encoding="utf-8") as openfile:
        result_dict_object = json.load(openfile)
        results_objectives_list = np.array(
        [[y[1][key] for key in result_dict_object["optimization_objectives"]]
        for y in [item for item in result_dict_object["results"]]])

        ref_point = np.max(results_objectives_list, axis=0)

        hv = HV(ref_point=ref_point)
        print("hyper volume")
        print(hv(results_objectives_list))


#directorycontent = os.listdir(path=results_directory)
#for _, item in enumerate(directorycontent):
#    print(item)



results: 
[[1.4369 1.4798]
 [1.4365 1.4735]
 [1.4214 1.4672]
 [1.1268 1.2588]
 [1.3906 1.4104]
 [1.2826 1.322 ]
 [1.3312 1.3725]
 [1.408  1.4609]
 [1.5028 1.5682]
 [1.317  1.3662]
 [0.766  1.1957]
 [1.0802 1.2462]
 [1.2301 1.2904]
 [1.1979 1.2778]
 [1.1548 1.2652]
 [0.7582 1.1894]
 [1.4431 1.4924]
 [1.5164 1.5871]
 [1.4029 1.4545]
 [0.9875 1.202 ]
 [1.2973 1.3598]
 [1.072  1.2273]
 [1.4555 1.5114]
 [1.4447 1.4987]
 [1.1259 1.2525]
 [1.541  1.5997]
 [1.383  1.3788]
 [1.2374 1.2967]
 [1.1644 1.2715]
 [1.028  1.2146]
 [1.5331 1.5934]
 [1.2703 1.303 ]
 [1.3985 1.4482]
 [1.4697 1.5303]
 [1.08   1.2336]
 [1.4916 1.5429]
 [1.5557 1.6061]]
ref points:
[1.5557 1.6061]
hyper volume
0.3323182500000001
